# Project: Investigate a Dataset - Medical Appointment No Shows

## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#wrangling">Data Wrangling</a></li>
<li><a href="#eda">Exploratory Data Analysis</a></li>
<li><a href="#conclusions">Conclusions</a></li>
</ul>

<a id='intro'></a>

## Introduction

### Dataset Description  
The dataset **"Medical Appointment No Shows"** contains records of **110,527** medical appointments, with information on whether or not the patient attended their scheduled appointment. The dataset aims to identify factors contributing to patient no-shows, which account for approximately **30%** of scheduled appointments.

## Data Overview  
The dataset includes **14 variables**, each representing different aspects of patient appointments and demographics. Below is a brief description of each column:

- **PatientId** – Unique identifier for each patient.
- **AppointmentID** – Unique identifier for each appointment.
- **Gender** – Patient’s gender (Male/Female).
- **DataMarcacaoConsulta** – Date of the scheduled appointment.
- **DataAgendamento** – Date when the appointment was booked.
- **Age** – Patient’s age.
- **Neighbourhood** – The location where the appointment is scheduled.
- **Scholarship** – Indicates whether the patient is enrolled in the [Bolsa Família](https://en.wikipedia.org/wiki/Bolsa_Fam%C3%ADlia) program (a social welfare program in Brazil).
- **Hipertension** – Indicates whether the patient has hypertension (True/False).
- **Diabetes** – Indicates whether the patient has diabetes (True/False).
- **Alcoholism** – Indicates whether the patient has a history of alcoholism (True/False).
- **Handcap** – Indicates whether the patient has any disability (True/False).
- **SMS_received** – Number of reminder messages sent to the patient.
- **No-show** – **Target variable** (True/False) – whether the patient attended the appointment or not.



### Question(s) for Analysis

1- What factors influence whether a patient will miss their scheduled medical appointment?

2- How does age affect the probability of missing an appointment?

 

In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

<a id='wrangling'></a>
## Data Wrangling



### Data Cleaning

In [ ]:
# Load the dataset
df = pd.read_csv("medical_appointment_no_shows.csv")
df.shape

The dataset consists of **110,527** recorde (rows) and **14 columns**

In [ ]:
df.head()

Here we can take an overview of the data shape and values 

In [ ]:
df.info()

#### 1. Data Types and Memory Usage
- The dataset consists of **110,527 rows and 14 columns**, using **11.8+ MB** of memory.
- The data includes **a mix of numerical (`int64`, `float64`) and (`object`) types**.
- **No missing values** are present; all columns have 110,527 non-null entries.

#### 2. Necessary Data Type Conversions
- `PatientId` is stored as **float64**, but since IDs don’t require decimals, it should be converted to **int64**.
- `ScheduledDay` and `AppointmentDay` are **object** types instead of datetime. They need to be converted for proper date analysis.
- **Many categorical variables** are stored as `object` (strings), but should be converted to `category` type to optimize memory usage.


In [ ]:
df.describe()

The **minimum age is -1**, which is **impossible** and should be corrected

In [ ]:
df.nunique()



- **62,299 unique patients** (`PatientId`), indicating that some patients have multiple appointments.
- **110,527 unique appointments** (`AppointmentID`), confirming each row represents an individual appointment.
- **81 different neighborhoods**, meaning appointments were scheduled across multiple locations.


- **Categorical Variables to Convert**:
  - `Gender` – Binary category (Male/Female).
  - `Neighbourhood` – 81 unique values, representing appointment locations.
  - `Scholarship`, `Hypertension`, `Diabetes`, `Alcoholism`, `SMS_received`, `No-show` – All **binary** (0 = No, 1 = Yes).
  - `Handcap` – Contains **five unique values (0, 1, 2, 3, 4)**

In [ ]:
df['Handcap'].value_counts()

The majority of patients (**108,286**) have **no disability (`0`)**, while a smaller number have disabilities ranging from **1 to 4**. Since higher values are rare, we might consider grouping them into a **binary category** (`0 = No disability`, `1+ = Has disability`).

In [ ]:
df['Age'].value_counts()

- The dataset contains **104 unique age values**, ranging from **-1 to 115**.
- The most common age is **0 (3,539 occurrences)**, which likely represents newborns or infants.
- Ages **1, 49, 52, and 53** are also frequent, each with over **1,500 occurrences**.
- The presence of **negative values (-1)** is an anomaly and should be removed or corrected.

Dataset for analysis, the following cleaning steps are required:
1. **Convert date columns (`ScheduledDay`, `AppointmentDay`) to datetime format** for easier analysis.


In [ ]:
# Convert date columns to datetime format
df["ScheduledDay"] = pd.to_datetime(df["ScheduledDay"])
df["AppointmentDay"] = pd.to_datetime(df["AppointmentDay"])

2. **Change `PatientId` from `float64` to `int64`**, as patient IDs do not require decimal points.


In [ ]:
# Convert PatientId to integer
df["PatientId"] = df["PatientId"].astype("int64")


3. **Convert categorical variables (`Gender`, `Neighbourhood`, etc.) to category type** for optimized performance.


#### Categorical Variables
The following columns contain categorical data and should be converted to the **category** data type for optimized storage and analysis:
1. **Gender** – Binary category (Male/Female).
2. **Neighbourhood** – Contains **81 unique values**, representing different locations.
3. **Scholarship** – Binary (0 = No, 1 = Yes).
4. **Hypertension** – Binary (0 = No, 1 = Yes).
5. **Diabetes** – Binary (0 = No, 1 = Yes).
6. **Alcoholism** – Binary (0 = No, 1 = Yes).
7. **Handcap** – Multiple levels (0, 1, 2, 3, 4) but can be simplified.
8. **SMS_received** – Binary (0 = No, 1 = Yes).
9. **No-show** – Binary (0 = Show, 1 = No-show), though it might be stored as a string and needs conversion.


In [ ]:
# Convert categorical variables to category type
categorical_cols = ["Gender","Handcap", "Neighbourhood", "Scholarship", "Hipertension", "Diabetes", "Alcoholism", "SMS_received", "No-show"]
df[categorical_cols] = df[categorical_cols].astype("category")

4. **Handle the incorrect `Age` values** by removing or correcting negative values.

In [ ]:
# Handle incorrect age values (remove negative ages)
df = df[df["Age"] >= 0]

#### Confirm changes

In [ ]:
# Confirm changes
df.info()

### **DataFrame Structure After Cleaning**

- **Total Entries**: The dataset now contains **110,526 rows**, meaning one row removed which is invalid `Age` value `-1`).

- **Memory Usage**: The dataset now uses **6.0 MB**, which is significantly reduced from 11.8+ MB after converting categorical columns.

### **Data Type Optimizations**
- **`PatientId`** is now stored as an **integer (`int64`)**, which is more appropriate than `float64`.
- **`ScheduledDay`** and **`AppointmentDay`** have been properly converted to **datetime64**, allowing for easier date operations.
- **8 categorical columns** (`Gender`, `Neighbourhood`, `Scholarship`, `Hypertension`, `Diabetes`, `Alcoholism`, `SMS_received`, `No-show`) were converted to **category**, improving performance.


In [ ]:
df.nunique()

<a id='eda'></a>
## Exploratory Data Analysis

### Research Question 1 

What factors influence whether a patient will miss their scheduled medical appointment?

### Research Question 2
How does age affect the probability of missing an appointment?

In [ ]:
# Plotting the distribution of No-show
plt.figure(figsize=(6, 4))
sns.countplot(x='No-show', data=df, palette="Set2")
plt.title("Distribution of No-show")
plt.xlabel("No-show (Yes/No)")
plt.ylabel("Count")
plt.show()


### Distribution of No-show

In this plot, I visualized the distribution of the **No-show** variable, which indicates whether a patient attended or missed their scheduled appointment. The plot shows that the majority of patients attended their appointments (around **90,000**), while a smaller portion (around **21,000**) missed their appointments. This gives us an overall sense of the proportion of no-shows in the dataset (about **30% no-shows** and **70% attendance**).


In [ ]:
# Create histograms for Age vs. No-show
plt.figure(figsize=(10, 6))

# Histogram for patients who attended (No-show == 'No')
sns.histplot(df[df['No-show'] == 'No']['Age'], color='blue', label='Attended', bins=30, kde=False, stat='count', alpha=0.6)

# Histogram for patients who did not attend (No-show == 'Yes')
sns.histplot(df[df['No-show'] == 'Yes']['Age'], color='red', label='No-show', bins=30, kde=False, stat='count', alpha=0.6)

# Calculate mean and standard deviation using NumPy
mean_age = np.mean(df[df['No-show'] == 'No']['Age'])
std_age = np.std(df[df['No-show'] == 'No']['Age'], ddof=1)  # ddof=1 for sample standard deviation

# Plot mean and standard deviation lines
plt.axvline(mean_age, color='black', linestyle='--', label=f'Attended Mean')
plt.axvline(mean_age + std_age, color='green', linestyle=':', label=f'Attended SD')
plt.axvline(mean_age - std_age, color='green', linestyle=':')

# Customize the plot
plt.title("Age Distribution by No-show Status (Count of Patients)")
plt.xlabel("Age")
plt.ylabel("Count of Patients")
plt.legend()
plt.show()




### Age Distribution by No-show Status

This plot compares the **age distribution** for patients who attended their appointments (**blue**) and those who missed them (**red**). I also added vertical lines representing the **mean age** (black dashed line) and **one standard deviation** above and below the mean (green dotted lines) for the attended group.

- The **mean age** for patients who attended is **37 years**, with a **standard deviation of 12 years and 62 years**.
- The plot shows that most of the attended patients fall within a central age range, while the no-show group shows a wider spread of ages.

This analysis helps to understand how **age** may influence the likelihood of attending or missing an appointment.


In [ ]:
plt.figure(figsize=(8, 5))
sns.boxplot(x=df['Age'], color="lightblue")
plt.title("Boxplot of Age")
plt.xlabel("Age")
plt.show()


In [ ]:
- The box represents the **interquartile range (IQR)** (25th to 75th percentile).  
- The line inside the box is the **median age**.  
- The whiskers extend to **1.5 times the IQR**.  
- Any points outside the whiskers are **potential outliers**, which may indicate extreme values in the dataset.  


<a id='conclusions'></a>
## Conclusions

### **Data Wrangling**  
- Cleaned and optimized data by fixing types, handling incorrect values, and converting categorical variables.  

### **Exploratory Data Analysis**  
- **70% of patients attended, 30% were no-shows**.  
- **Average age of attendees: 37 years (SD: 12)**.  
- **Age impacts attendance**, with variations across age groups.  
- **Box plot shows outliers**, indicating extreme values.  

### **Key Takeaways**  
- No-shows are a significant issue.  
- Age plays a role in appointment attendance.  
- Data is now ready for deeper analysis.  




In [ ]:
# Running this cell will execute a bash command to convert this notebook to an .html file
!python -m nbconvert --to html Investigate_a_Dataset.ipynb